# US Border Crossing Entry Data Analysis

## Portfolio Project: Comprehensive Analysis of US Border Crossings

This analysis examines border crossing patterns between the US-Canada and US-Mexico borders, focusing on different transportation modes, temporal trends, and geographical patterns.

### Dataset Overview
- **Source**: US Border Crossing Entry Data
- **Records**: ~403,000 entries
- **Time Period**: 2024 data
- **Coverage**: US-Canada and US-Mexico border crossings
- **Measures**: Various transportation modes and passenger/cargo types

## 1. Data Loading and Initial Exploration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load the dataset
df = pd.read_csv('Border_Crossing_Entry_Data.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nColumn Names: {list(df.columns)}")
print(f"\nData Types:")
print(df.dtypes)

In [ ]:
# Display first few rows and basic info
display(df.head(10))

print("\n=== BASIC DATASET INFO ===")
df.info()

print("\n=== MISSING VALUES ===")
print(df.isnull().sum())

print("\n=== UNIQUE VALUES PER COLUMN ===")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

## 2. Data Cleaning and Preprocessing

In [ ]:
# Create a copy for processing
df_clean = df.copy()

# Convert Date to datetime
df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%b %Y')

# Extract year and month for analysis
df_clean['Year'] = df_clean['Date'].dt.year
df_clean['Month'] = df_clean['Date'].dt.month
df_clean['Month_Name'] = df_clean['Date'].dt.strftime('%B')

# Clean and standardize categorical variables
df_clean['Border'] = df_clean['Border'].str.strip()
df_clean['State'] = df_clean['State'].str.strip()
df_clean['Measure'] = df_clean['Measure'].str.strip()

# Handle any missing or invalid values
print(f"Records with missing values: {df_clean.isnull().sum().sum()}")
print(f"Records with zero or negative values: {(df_clean['Value'] <= 0).sum()}")

# Remove records with zero or negative values for meaningful analysis
df_clean = df_clean[df_clean['Value'] > 0]

print(f"\nFinal dataset shape after cleaning: {df_clean.shape}")

# Display unique values for key categorical variables
print("\n=== UNIQUE VALUES AFTER CLEANING ===")
print(f"Borders: {df_clean['Border'].unique()}")
print(f"States: {sorted(df_clean['State'].unique())}")
print(f"Measures: {sorted(df_clean['Measure'].unique())}")
print(f"Date range: {df_clean['Date'].min()} to {df_clean['Date'].max()}")

## 3. Exploratory Data Analysis (EDA)

### 3.1 Overall Statistics and Distribution

In [ ]:
# Basic statistics
print("=== DESCRIPTIVE STATISTICS ===")
print(df_clean['Value'].describe())

# Distribution of values by border
border_stats = df_clean.groupby('Border')['Value'].agg(['count', 'sum', 'mean', 'median', 'std']).round(2)
print("\n=== STATISTICS BY BORDER ===")
display(border_stats)

# Distribution by measure type
measure_stats = df_clean.groupby('Measure')['Value'].agg(['count', 'sum', 'mean', 'median']).sort_values('sum', ascending=False)
print("\n=== TOP 10 MEASURES BY TOTAL VOLUME ===")
display(measure_stats.head(10))

### 3.2 Border Comparison Analysis

In [ ]:
# Create comprehensive border comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Total volume by border
border_totals = df_clean.groupby('Border')['Value'].sum().sort_values(ascending=False)
axes[0,0].bar(border_totals.index, border_totals.values, color=['#2E8B57', '#CD853F'])
axes[0,0].set_title('Total Border Crossing Volume', fontsize=14, fontweight='bold')
axes[0,0].set_ylabel('Total Crossings')
axes[0,0].tick_params(axis='x', rotation=45)
for i, v in enumerate(border_totals.values):
    axes[0,0].text(i, v + v*0.01, f'{v:,.0f}', ha='center', fontweight='bold')

# 2. Average volume by border
border_avg = df_clean.groupby('Border')['Value'].mean().sort_values(ascending=False)
axes[0,1].bar(border_avg.index, border_avg.values, color=['#4682B4', '#DAA520'])
axes[0,1].set_title('Average Border Crossing Volume', fontsize=14, fontweight='bold')
axes[0,1].set_ylabel('Average Crossings')
axes[0,1].tick_params(axis='x', rotation=45)
for i, v in enumerate(border_avg.values):
    axes[0,1].text(i, v + v*0.01, f'{v:,.0f}', ha='center', fontweight='bold')

# 3. Number of ports by border
ports_by_border = df_clean.groupby('Border')['Port Name'].nunique().sort_values(ascending=False)
axes[1,0].bar(ports_by_border.index, ports_by_border.values, color=['#8B4513', '#228B22'])
axes[1,0].set_title('Number of Active Ports by Border', fontsize=14, fontweight='bold')
axes[1,0].set_ylabel('Number of Ports')
axes[1,0].tick_params(axis='x', rotation=45)
for i, v in enumerate(ports_by_border.values):
    axes[1,0].text(i, v + v*0.01, f'{v}', ha='center', fontweight='bold')

# 4. Volume distribution (box plot)
df_clean.boxplot(column='Value', by='Border', ax=axes[1,1])
axes[1,1].set_title('Volume Distribution by Border', fontsize=14, fontweight='bold')
axes[1,1].set_ylabel('Crossing Volume (log scale)')
axes[1,1].set_yscale('log')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Print summary statistics
print("=== BORDER COMPARISON SUMMARY ===")
print(f"US-Mexico Border: {border_totals['US-Mexico Border']:,.0f} total crossings ({border_totals['US-Mexico Border']/border_totals.sum()*100:.1f}%)")
print(f"US-Canada Border: {border_totals['US-Canada Border']:,.0f} total crossings ({border_totals['US-Canada Border']/border_totals.sum()*100:.1f}%)")
print(f"\nUS-Mexico has {border_totals['US-Mexico Border']/border_totals['US-Canada Border']:.1f}x more total crossings than US-Canada")

### 3.3 Transportation Mode Analysis

In [ ]:
# Analyze different transportation modes
transport_analysis = df_clean.groupby(['Border', 'Measure'])['Value'].sum().reset_index()
transport_pivot = transport_analysis.pivot(index='Measure', columns='Border', values='Value').fillna(0)

# Calculate totals for sorting
transport_pivot['Total'] = transport_pivot.sum(axis=1)
transport_pivot = transport_pivot.sort_values('Total', ascending=False)

# Create stacked bar chart
fig, ax = plt.subplots(figsize=(16, 10))

# Top 15 measures for readability
top_measures = transport_pivot.head(15)

x = np.arange(len(top_measures))
width = 0.6

p1 = ax.bar(x, top_measures['US-Canada Border'], width, label='US-Canada Border', color='#2E8B57', alpha=0.8)
p2 = ax.bar(x, top_measures['US-Mexico Border'], width, bottom=top_measures['US-Canada Border'], 
           label='US-Mexico Border', color='#CD853F', alpha=0.8)

ax.set_title('Border Crossings by Transportation Mode (Top 15)', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Transportation Mode', fontsize=12, fontweight='bold')
ax.set_ylabel('Total Crossings', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(top_measures.index, rotation=45, ha='right')
ax.legend(loc='upper right')

# Add value labels on bars
for i, (idx, row) in enumerate(top_measures.iterrows()):
    total = row['Total']
    if total > 1000000:
        ax.text(i, total + total*0.02, f'{total/1000000:.1f}M', ha='center', fontweight='bold')
    elif total > 1000:
        ax.text(i, total + total*0.02, f'{total/1000:.0f}K', ha='center', fontweight='bold')
    else:
        ax.text(i, total + total*0.02, f'{total:.0f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Print top transportation modes
print("=== TOP 10 TRANSPORTATION MODES ===")
for i, (measure, row) in enumerate(transport_pivot.head(10).iterrows(), 1):
    print(f"{i:2}. {measure:30} | Total: {row['Total']:>10,.0f} | Canada: {row['US-Canada Border']:>10,.0f} | Mexico: {row['US-Mexico Border']:>10,.0f}")

### 3.4 Temporal Analysis

In [ ]:
# Monthly trends analysis
monthly_data = df_clean.groupby(['Month_Name', 'Border'])['Value'].sum().reset_index()
monthly_pivot = monthly_data.pivot(index='Month_Name', columns='Border', values='Value')

# Reorder months
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_pivot = monthly_pivot.reindex([m for m in month_order if m in monthly_pivot.index])

# Create monthly trends plot
fig, axes = plt.subplots(2, 1, figsize=(15, 12))

# Line plot for trends
axes[0].plot(monthly_pivot.index, monthly_pivot['US-Canada Border'], 
            marker='o', linewidth=3, label='US-Canada Border', color='#2E8B57')
axes[0].plot(monthly_pivot.index, monthly_pivot['US-Mexico Border'], 
            marker='s', linewidth=3, label='US-Mexico Border', color='#CD853F')
axes[0].set_title('Monthly Border Crossing Trends', fontsize=16, fontweight='bold')
axes[0].set_ylabel('Total Crossings', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Stacked bar chart
monthly_pivot.plot(kind='bar', ax=axes[1], color=['#2E8B57', '#CD853F'], alpha=0.8)
axes[1].set_title('Monthly Border Crossings (Stacked)', fontsize=16, fontweight='bold')
axes[1].set_ylabel('Total Crossings', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Month', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=12)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Print monthly statistics
print("=== MONTHLY STATISTICS ===")
monthly_total = monthly_pivot.sum(axis=1).sort_values(ascending=False)
print("Busiest months (total crossings):")
for month, total in monthly_total.head(5).items():
    print(f"  {month}: {total:,.0f}")

print("\nQuietest months (total crossings):")
for month, total in monthly_total.tail(3).items():
    print(f"  {month}: {total:,.0f}")

### 3.5 Geographic Analysis

In [ ]:
# State-level analysis
state_analysis = df_clean.groupby(['State', 'Border'])['Value'].agg(['sum', 'count', 'mean']).round(0)
state_totals = df_clean.groupby('State')['Value'].sum().sort_values(ascending=False)

# Top states by total crossings
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# 1. Top 15 states by total volume
top_states = state_totals.head(15)
colors = plt.cm.Set3(np.linspace(0, 1, len(top_states)))
bars = axes[0,0].bar(range(len(top_states)), top_states.values, color=colors)
axes[0,0].set_title('Top 15 States by Total Border Crossings', fontsize=14, fontweight='bold')
axes[0,0].set_ylabel('Total Crossings')
axes[0,0].set_xticks(range(len(top_states)))
axes[0,0].set_xticklabels(top_states.index, rotation=45, ha='right')

# Add value labels
for i, v in enumerate(top_states.values):
    if v > 1000000:
        axes[0,0].text(i, v + v*0.01, f'{v/1000000:.1f}M', ha='center', fontweight='bold', fontsize=9)
    else:
        axes[0,0].text(i, v + v*0.01, f'{v/1000:.0f}K', ha='center', fontweight='bold', fontsize=9)

# 2. Border distribution by state
border_state = df_clean.groupby(['State', 'Border'])['Value'].sum().reset_index()
canada_states = border_state[border_state['Border'] == 'US-Canada Border']['State'].nunique()
mexico_states = border_state[border_state['Border'] == 'US-Mexico Border']['State'].nunique()

axes[0,1].pie([canada_states, mexico_states], labels=['US-Canada States', 'US-Mexico States'], 
              autopct='%1.0f', colors=['#2E8B57', '#CD853F'])
axes[0,1].set_title('Number of Border States', fontsize=14, fontweight='bold')

# 3. Top ports by volume
port_totals = df_clean.groupby(['Port Name', 'State'])['Value'].sum().sort_values(ascending=False).head(15)
port_labels = [f"{port}\n({state})" for port, state in port_totals.index]
axes[1,0].barh(range(len(port_totals)), port_totals.values, color='skyblue')
axes[1,0].set_title('Top 15 Ports by Total Volume', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Total Crossings')
axes[1,0].set_yticks(range(len(port_totals)))
axes[1,0].set_yticklabels(port_labels, fontsize=10)
axes[1,0].invert_yaxis()

# 4. Average crossings by state
state_avg = df_clean.groupby('State')['Value'].mean().sort_values(ascending=False).head(10)
axes[1,1].bar(range(len(state_avg)), state_avg.values, color='lightcoral')
axes[1,1].set_title('Top 10 States by Average Crossing Volume', fontsize=14, fontweight='bold')
axes[1,1].set_ylabel('Average Crossings')
axes[1,1].set_xticks(range(len(state_avg)))
axes[1,1].set_xticklabels(state_avg.index, rotation=45, ha='right')

plt.tight_layout()
plt.show()

# Print geographic insights
print("=== GEOGRAPHIC INSIGHTS ===")
print(f"Total number of states with border crossings: {df_clean['State'].nunique()}")
print(f"Total number of border ports: {df_clean['Port Name'].nunique()}")
print(f"States on US-Canada border: {canada_states}")
print(f"States on US-Mexico border: {mexico_states}")

print("\nTop 5 busiest ports:")
for i, ((port, state), volume) in enumerate(port_totals.head(5).items(), 1):
    print(f"  {i}. {port}, {state}: {volume:,.0f} crossings")

## 4. Advanced Analysis and Insights

### 4.1 Commercial vs Personal Traffic Analysis

In [ ]:
# Categorize measures into commercial and personal traffic
commercial_keywords = ['truck', 'rail', 'container', 'cargo']
personal_keywords = ['personal', 'pedestrian', 'bus passenger']

def categorize_traffic(measure):
    measure_lower = measure.lower()
    if any(keyword in measure_lower for keyword in commercial_keywords):
        return 'Commercial'
    elif any(keyword in measure_lower for keyword in personal_keywords):
        return 'Personal'
    elif 'bus' in measure_lower and 'passenger' not in measure_lower:
        return 'Commercial'
    else:
        return 'Other'

df_clean['Traffic_Category'] = df_clean['Measure'].apply(categorize_traffic)

# Analysis by traffic category
traffic_analysis = df_clean.groupby(['Border', 'Traffic_Category'])['Value'].sum().reset_index()
traffic_pivot = traffic_analysis.pivot(index='Traffic_Category', columns='Border', values='Value').fillna(0)

# Create comprehensive traffic analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Traffic category distribution
traffic_total = df_clean.groupby('Traffic_Category')['Value'].sum().sort_values(ascending=False)
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
wedges, texts, autotexts = axes[0,0].pie(traffic_total.values, labels=traffic_total.index, autopct='%1.1f%%', 
                                        colors=colors, startangle=90)
axes[0,0].set_title('Traffic Distribution by Category', fontsize=14, fontweight='bold')

# 2. Traffic by border
traffic_pivot.plot(kind='bar', ax=axes[0,1], color=['#2E8B57', '#CD853F'])
axes[0,1].set_title('Traffic Categories by Border', fontsize=14, fontweight='bold')
axes[0,1].set_ylabel('Total Crossings')
axes[0,1].legend(title='Border')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Monthly trends by traffic category
monthly_traffic = df_clean.groupby(['Month_Name', 'Traffic_Category'])['Value'].sum().reset_index()
monthly_traffic_pivot = monthly_traffic.pivot(index='Month_Name', columns='Traffic_Category', values='Value').fillna(0)
monthly_traffic_pivot = monthly_traffic_pivot.reindex([m for m in month_order if m in monthly_traffic_pivot.index])

for category, color in zip(monthly_traffic_pivot.columns, colors):
    if category in monthly_traffic_pivot.columns:
        axes[1,0].plot(monthly_traffic_pivot.index, monthly_traffic_pivot[category], 
                      marker='o', label=category, linewidth=2, color=color)

axes[1,0].set_title('Monthly Trends by Traffic Category', fontsize=14, fontweight='bold')
axes[1,0].set_ylabel('Total Crossings')
axes[1,0].legend()
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].grid(True, alpha=0.3)

# 4. Top measures in each category
commercial_measures = df_clean[df_clean['Traffic_Category'] == 'Commercial'].groupby('Measure')['Value'].sum().head(5)
personal_measures = df_clean[df_clean['Traffic_Category'] == 'Personal'].groupby('Measure')['Value'].sum().head(5)

y_pos = np.arange(len(commercial_measures))
axes[1,1].barh(y_pos, commercial_measures.values, alpha=0.7, color='#FF6B6B', label='Commercial')
axes[1,1].set_yticks(y_pos)
axes[1,1].set_yticklabels(commercial_measures.index, fontsize=10)
axes[1,1].set_title('Top 5 Commercial Traffic Measures', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Total Crossings')

plt.tight_layout()
plt.show()

# Print traffic insights
print("=== TRAFFIC CATEGORY ANALYSIS ===")
for category, total in traffic_total.items():
    percentage = (total / traffic_total.sum()) * 100
    print(f"{category}: {total:,.0f} crossings ({percentage:.1f}%)")

print("\n=== COMMERCIAL VS PERSONAL BY BORDER ===")
for border in traffic_pivot.columns:
    commercial = traffic_pivot.loc['Commercial', border]
    personal = traffic_pivot.loc['Personal', border]
    ratio = commercial / personal if personal > 0 else 0
    print(f"{border}:")
    print(f"  Commercial: {commercial:,.0f}")
    print(f"  Personal: {personal:,.0f}")
    print(f"  Commercial/Personal Ratio: {ratio:.2f}")

### 4.2 Statistical Analysis and Correlations

In [ ]:
# Create correlation analysis
# Prepare data for correlation analysis
correlation_data = df_clean.pivot_table(
    index=['Port Name', 'State', 'Border', 'Month'], 
    columns='Measure', 
    values='Value', 
    fill_value=0
).reset_index()

# Select top measures for correlation
top_measures = df_clean.groupby('Measure')['Value'].sum().nlargest(10).index
corr_subset = correlation_data[top_measures]

# Calculate correlation matrix
correlation_matrix = corr_subset.corr()

# Create heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, mask=mask, cbar_kws={'shrink': 0.8})
plt.title('Correlation Matrix: Top 10 Transportation Measures', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Statistical tests
from scipy import stats

# Compare border crossing volumes
canada_values = df_clean[df_clean['Border'] == 'US-Canada Border']['Value']
mexico_values = df_clean[df_clean['Border'] == 'US-Mexico Border']['Value']

# Mann-Whitney U test (non-parametric)
statistic, p_value = stats.mannwhitneyu(canada_values, mexico_values, alternative='two-sided')

print("=== STATISTICAL ANALYSIS ===")
print(f"Mann-Whitney U test comparing border volumes:")
print(f"  Statistic: {statistic:,.0f}")
print(f"  P-value: {p_value:.2e}")
print(f"  Significant difference: {'Yes' if p_value < 0.05 else 'No'}")

# Descriptive statistics comparison
print("\n=== DESCRIPTIVE STATISTICS BY BORDER ===")
stats_comparison = df_clean.groupby('Border')['Value'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max', 
    lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)
]).round(2)
stats_comparison.columns = ['Count', 'Mean', 'Median', 'Std Dev', 'Min', 'Max', 'Q1', 'Q3']
display(stats_comparison)

# Seasonal analysis
seasonal_stats = df_clean.groupby(['Border', 'Month_Name'])['Value'].sum().reset_index()
seasonal_pivot = seasonal_stats.pivot(index='Month_Name', columns='Border', values='Value')
seasonal_corr = seasonal_pivot.corr().iloc[0, 1]

print(f"\n=== SEASONAL CORRELATION ===")
print(f"Correlation between US-Canada and US-Mexico monthly patterns: {seasonal_corr:.3f}")
print(f"Interpretation: {'Strong positive' if seasonal_corr > 0.7 else 'Moderate positive' if seasonal_corr > 0.3 else 'Weak' if seasonal_corr > 0.1 else 'Very weak'} correlation")

### 4.3 Interactive Geographic Visualization

In [ ]:
# Create geographic data for mapping
geo_data = df_clean.groupby(['Port Name', 'State', 'Border', 'Latitude', 'Longitude'])['Value'].sum().reset_index()

# Create interactive map using plotly
fig = px.scatter_mapbox(geo_data, 
                       lat='Latitude', 
                       lon='Longitude',
                       size='Value',
                       color='Border',
                       hover_name='Port Name',
                       hover_data={'State': True, 'Value': ':,.0f', 'Latitude': False, 'Longitude': False},
                       color_discrete_map={'US-Canada Border': '#2E8B57', 'US-Mexico Border': '#CD853F'},
                       size_max=30,
                       zoom=3,
                       title='US Border Crossings by Port Location')

fig.update_layout(mapbox_style="open-street-map",
                 height=600,
                 title_font_size=16)

fig.show()

# Alternative static map using matplotlib
fig, ax = plt.subplots(figsize=(15, 10))

# Separate data by border
canada_ports = geo_data[geo_data['Border'] == 'US-Canada Border']
mexico_ports = geo_data[geo_data['Border'] == 'US-Mexico Border']

# Create scatter plot
scatter1 = ax.scatter(canada_ports['Longitude'], canada_ports['Latitude'], 
                     s=canada_ports['Value']/50000, alpha=0.6, 
                     c='#2E8B57', label='US-Canada Border')

scatter2 = ax.scatter(mexico_ports['Longitude'], mexico_ports['Latitude'], 
                     s=mexico_ports['Value']/50000, alpha=0.6, 
                     c='#CD853F', label='US-Mexico Border')

ax.set_xlabel('Longitude', fontsize=12, fontweight='bold')
ax.set_ylabel('Latitude', fontsize=12, fontweight='bold')
ax.set_title('Geographic Distribution of Border Ports\n(Size proportional to crossing volume)', 
             fontsize=16, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

# Add annotations for top ports
top_ports = geo_data.nlargest(10, 'Value')
for idx, port in top_ports.iterrows():
    ax.annotate(f"{port['Port Name']}, {port['State']}", 
               (port['Longitude'], port['Latitude']),
               xytext=(5, 5), textcoords='offset points',
               fontsize=8, alpha=0.8)

plt.tight_layout()
plt.show()

print("=== GEOGRAPHIC DISTRIBUTION ===")
print(f"Northernmost port: {geo_data.loc[geo_data['Latitude'].idxmax(), 'Port Name']}, {geo_data.loc[geo_data['Latitude'].idxmax(), 'State']} ({geo_data['Latitude'].max():.3f}°N)")
print(f"Southernmost port: {geo_data.loc[geo_data['Latitude'].idxmin(), 'Port Name']}, {geo_data.loc[geo_data['Latitude'].idxmin(), 'State']} ({geo_data['Latitude'].min():.3f}°N)")
print(f"Easternmost port: {geo_data.loc[geo_data['Longitude'].idxmax(), 'Port Name']}, {geo_data.loc[geo_data['Longitude'].idxmax(), 'State']} ({geo_data['Longitude'].max():.3f}°W)")
print(f"Westernmost port: {geo_data.loc[geo_data['Longitude'].idxmin(), 'Port Name']}, {geo_data.loc[geo_data['Longitude'].idxmin(), 'State']} ({geo_data['Longitude'].min():.3f}°W)")

## 5. Key Findings and Business Insights

In [ ]:
# Generate comprehensive summary statistics
total_crossings = df_clean['Value'].sum()
total_ports = df_clean['Port Name'].nunique()
total_states = df_clean['State'].nunique()
date_range = f"{df_clean['Date'].min().strftime('%B %Y')} to {df_clean['Date'].max().strftime('%B %Y')}"

# Border comparison
border_summary = df_clean.groupby('Border')['Value'].agg(['sum', 'mean', 'count']).round(0)
us_mexico_total = border_summary.loc['US-Mexico Border', 'sum']
us_canada_total = border_summary.loc['US-Canada Border', 'sum']
mexico_dominance = us_mexico_total / us_canada_total

# Top performers
busiest_port = df_clean.groupby(['Port Name', 'State'])['Value'].sum().idxmax()
busiest_port_volume = df_clean.groupby(['Port Name', 'State'])['Value'].sum().max()

busiest_state = df_clean.groupby('State')['Value'].sum().idxmax()
busiest_state_volume = df_clean.groupby('State')['Value'].sum().max()

top_measure = df_clean.groupby('Measure')['Value'].sum().idxmax()
top_measure_volume = df_clean.groupby('Measure')['Value'].sum().max()

busiest_month = monthly_pivot.sum(axis=1).idxmax()
busiest_month_volume = monthly_pivot.sum(axis=1).max()

# Create summary dashboard
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('US Border Crossing Analysis - Executive Summary Dashboard', fontsize=20, fontweight='bold')

# 1. Total volume by border (donut chart)
sizes = [us_canada_total, us_mexico_total]
labels = ['US-Canada', 'US-Mexico']
colors = ['#2E8B57', '#CD853F']
wedges, texts, autotexts = axes[0,0].pie(sizes, labels=labels, autopct='%1.1f%%', 
                                        colors=colors, startangle=90,
                                        wedgeprops=dict(width=0.5))
axes[0,0].set_title('Total Volume by Border\n(Donut Chart)', fontsize=14, fontweight='bold')

# 2. Monthly trend
monthly_total = monthly_pivot.sum(axis=1)
axes[0,1].plot(monthly_total.index, monthly_total.values, marker='o', linewidth=3, color='#4CAF50')
axes[0,1].set_title('Total Monthly Crossings\n(Combined Borders)', fontsize=14, fontweight='bold')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].grid(True, alpha=0.3)

# 3. Top 5 states
top_5_states = df_clean.groupby('State')['Value'].sum().nlargest(5)
axes[0,2].bar(range(len(top_5_states)), top_5_states.values, color='lightblue')
axes[0,2].set_title('Top 5 States by Volume', fontsize=14, fontweight='bold')
axes[0,2].set_xticks(range(len(top_5_states)))
axes[0,2].set_xticklabels(top_5_states.index, rotation=45)

# 4. Traffic category distribution
traffic_dist = df_clean.groupby('Traffic_Category')['Value'].sum()
axes[1,0].pie(traffic_dist.values, labels=traffic_dist.index, autopct='%1.1f%%', 
              colors=['#FF6B6B', '#4ECDC4', '#45B7D1'])
axes[1,0].set_title('Traffic Category\nDistribution', fontsize=14, fontweight='bold')

# 5. Top measures
top_5_measures = df_clean.groupby('Measure')['Value'].sum().nlargest(5)
axes[1,1].barh(range(len(top_5_measures)), top_5_measures.values, color='salmon')
axes[1,1].set_title('Top 5 Transportation Measures', fontsize=14, fontweight='bold')
axes[1,1].set_yticks(range(len(top_5_measures)))
axes[1,1].set_yticklabels(top_5_measures.index, fontsize=10)
axes[1,1].invert_yaxis()

# 6. Key metrics text
axes[1,2].text(0.1, 0.9, 'KEY METRICS', fontsize=16, fontweight='bold', transform=axes[1,2].transAxes)
metrics_text = f"""Total Crossings: {total_crossings:,.0f}
Active Ports: {total_ports}
States Involved: {total_states}
Time Period: {date_range}

Busiest Port:
{busiest_port[0]}, {busiest_port[1]}
({busiest_port_volume:,.0f} crossings)

Busiest State: {busiest_state}
({busiest_state_volume:,.0f} crossings)

Peak Month: {busiest_month}
({busiest_month_volume:,.0f} crossings)

Border Volume Ratio:
Mexico/Canada = {mexico_dominance:.1f}x"""

axes[1,2].text(0.1, 0.8, metrics_text, fontsize=11, transform=axes[1,2].transAxes, 
               verticalalignment='top', fontfamily='monospace')
axes[1,2].axis('off')

plt.tight_layout()
plt.show()

print("" + "="*80)
print("                    US BORDER CROSSING ANALYSIS - EXECUTIVE SUMMARY")
print("="*80)
print(f"Analysis Period: {date_range}")
print(f"Total Border Crossings Analyzed: {total_crossings:,.0f}")
print(f"Number of Border Ports: {total_ports}")
print(f"States with Border Activity: {total_states}")
print("\n" + "-"*80)
print("KEY FINDINGS:")
print("-"*80)

print(f"\n1. BORDER COMPARISON:")
print(f"   • US-Mexico Border dominates with {us_mexico_total:,.0f} crossings ({us_mexico_total/total_crossings*100:.1f}%)")
print(f"   • US-Canada Border accounts for {us_canada_total:,.0f} crossings ({us_canada_total/total_crossings*100:.1f}%)")
print(f"   • US-Mexico has {mexico_dominance:.1f}x more crossings than US-Canada")

print(f"\n2. TOP PERFORMERS:")
print(f"   • Busiest Port: {busiest_port[0]}, {busiest_port[1]} ({busiest_port_volume:,.0f} crossings)")
print(f"   • Busiest State: {busiest_state} ({busiest_state_volume:,.0f} crossings)")
print(f"   • Most Common Measure: {top_measure} ({top_measure_volume:,.0f} crossings)")
print(f"   • Peak Month: {busiest_month} ({busiest_month_volume:,.0f} crossings)")

print(f"\n3. TRAFFIC COMPOSITION:")
traffic_percentages = (df_clean.groupby('Traffic_Category')['Value'].sum() / total_crossings * 100).round(1)
for category, percentage in traffic_percentages.items():
    print(f"   • {category}: {percentage}%")

print(f"\n4. SEASONAL PATTERNS:")
monthly_variance = monthly_pivot.sum(axis=1)
peak_to_trough = monthly_variance.max() / monthly_variance.min()
print(f"   • Peak-to-trough ratio: {peak_to_trough:.1f}x")
print(f"   • Highest volume: {monthly_variance.idxmax()} ({monthly_variance.max():,.0f})")
print(f"   • Lowest volume: {monthly_variance.idxmin()} ({monthly_variance.min():,.0f})")

print(f"\n5. STATISTICAL INSIGHTS:")
print(f"   • Average crossing volume per record: {df_clean['Value'].mean():,.0f}")
print(f"   • Median crossing volume: {df_clean['Value'].median():,.0f}")
print(f"   • Standard deviation: {df_clean['Value'].std():,.0f}")
print(f"   • Coefficient of variation: {df_clean['Value'].std()/df_clean['Value'].mean():.2f}")

print("\n" + "="*80)

## 6. Recommendations and Future Analysis

### Business Recommendations:

1. **Resource Allocation**: Focus infrastructure investments on US-Mexico border given the 5.9x higher volume
2. **Seasonal Planning**: Prepare for higher volumes during peak months (March-April)
3. **Port Optimization**: Prioritize improvements at high-volume ports like those in Texas and California
4. **Commercial Traffic**: Develop specialized processing for truck and container traffic

### Future Analysis Opportunities:

1. **Predictive Modeling**: Develop models to forecast crossing volumes
2. **Economic Impact**: Correlate crossing data with economic indicators
3. **Security Analysis**: Analyze patterns for anomaly detection
4. **Capacity Planning**: Model optimal resource allocation across ports

### Technical Improvements:

1. **Real-time Dashboard**: Create live monitoring system
2. **Automated Reporting**: Generate regular insights
3. **Data Integration**: Combine with weather, economic, and policy data
4. **Machine Learning**: Implement clustering and classification models
